In [1]:
!wget https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv

--2021-11-17 05:18:11--  https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3858 (3.8K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.77K  --.-KB/s    in 0s      

2021-11-17 05:18:11 (39.0 MB/s) - ‘iris.csv’ saved [3858/3858]



In [2]:
import numpy as np
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset,DataLoader
import pandas as pd
from sklearn import preprocessing

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
class IrisDataset(Dataset):
    def __init__(self, file):
        self.data = pd.read_csv(file)
        input_features = self.data.values[:, 0:4].astype('float32')
        self.x_train = torch.from_numpy(input_features)
        
        label_encoder = preprocessing.LabelEncoder()
        self.data['species'] = label_encoder.fit_transform(self.data['species'])
        mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
        print(mapping)
        target_label = self.data['species'].values
        
        self.y_train = torch.from_numpy(target_label)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        return (self.x_train[idx],self.y_train[idx])

In [5]:
class MyDataset(Dataset):
    def __init__(self, file):
        self.data = pd.read_csv(file)

        input_features = self.data.values[:, 0:2].astype('float32')
        self.x_train = torch.from_numpy(input_features)
        
        target_label = self.data['y'].values
        self.y_train = torch.from_numpy(target_label)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        return (self.x_train[idx],self.y_train[idx])


In [8]:
class Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, output_dim)


    def forward(self, x):
        x = F.softmax(self.layer1(x))
        return x

In [6]:
dataset = IrisDataset('iris.csv')
iris_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

{'setosa': 0, 'versicolor': 1, 'virginica': 2}


In [7]:
mydataset = MyDataset('my_data.csv')
my_dataloader = DataLoader(mydataset, batch_size=4, shuffle=True)

In [13]:
iris_model = Model(4,3).to(device)
my_model = Model(2,2).to(device)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer_iris = torch.optim.Adam(iris_model.parameters(), lr=0.001)
optimizer_my = torch.optim.Adam(my_model.parameters(), lr=0.001)

# Train Iris

In [ ]:
epochs = 200

for epoch in range(1,epochs+1):
    print("EPOCH:",epoch,end=" ")
    running_loss = 0
    running_acc = 0
    
    for data,labels in iris_dataloader:
        data,labels = data.to(device),labels.to(device)
        optimizer_iris.zero_grad()
        output = iris_model.forward(data)
        loss = criterion(output,labels)
        
        result = torch.argmax(output,dim=1)
        running_loss+=loss.item()
        running_acc+=torch.mean((result==labels).type(torch.float))
        
        loss.backward()
        optimizer_iris.step()
    else:
        train_loss = running_loss/len(iris_dataloader)
        train_acc = running_acc/len(iris_dataloader)
        
        print("Training Loss: {:.3f}".format(train_loss),end=" ") 
       
        print("Train Accuracy: {:.2f}%".format(train_acc.item()*100))

In [18]:
iris_model.layer1.weight

Parameter containing:
tensor([[-0.0810,  1.1333, -1.0530, -0.9367],
        [-0.1015, -0.0743,  0.1437, -0.2491],
        [-0.3555, -0.2440,  0.4371,  0.7291]], requires_grad=True)

In [19]:
iris_model.layer1.bias

Parameter containing:
tensor([ 0.5717,  0.0635, -0.6569], requires_grad=True)

In [24]:
test_iris = torch.tensor([[1.2,4.1,3.1,1.1]])

In [25]:
iris_model(test_iris)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor([[0.5458, 0.1972, 0.2570]], grad_fn=<SoftmaxBackward0>)

# Manual iris model interpret

In [26]:
weight_ = iris_model.layer1.weight

bias_ = iris_model.layer1.bias

In [27]:
w = torch.transpose(weight_, 0,1)

In [28]:
w

tensor([[-0.0810, -0.1015, -0.3555],
        [ 1.1333, -0.0743, -0.2440],
        [-1.0530,  0.1437,  0.4371],
        [-0.9367, -0.2491,  0.7291]], grad_fn=<TransposeBackward0>)

In [30]:
z = torch.matmul(test_iris, w) + bias_

In [31]:
z

tensor([[ 0.8265, -0.1917,  0.0733]], grad_fn=<AddBackward0>)

In [32]:
F.softmax(z)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.5458, 0.1972, 0.2570]], grad_fn=<SoftmaxBackward0>)

# Train my

In [ ]:
epochs = 100

for epoch in range(1,epochs+1):
    print("EPOCH:",epoch,end=" ")
    running_loss = 0
    running_acc = 0
    
    for data,labels in my_dataloader:
        data,labels = data.to(device),labels.to(device)
        optimizer_my.zero_grad()
        output = my_model.forward(data)
        loss = criterion(output,labels)
        
        result = torch.argmax(output,dim=1)
        running_loss+=loss.item()
        running_acc+=torch.mean((result==labels).type(torch.float))
        
        loss.backward()
        optimizer_my.step()
    else:
        train_loss = running_loss/len(my_dataloader)
        train_acc = running_acc/len(my_dataloader)
        
        print("Training Loss: {:.3f}".format(train_loss),end=" ") 
       
        print("Train Accuracy: {:.2f}%".format(train_acc.item()*100))

In [44]:
test_my = torch.tensor([[1.2,1.1]])

In [45]:
my_model(test_my)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor([[0.2522, 0.7478]], grad_fn=<SoftmaxBackward0>)

In [33]:
my_model.layer1.weight

Parameter containing:
tensor([[-0.4598,  0.5194],
        [ 0.0734, -0.3372]], requires_grad=True)

In [34]:
my_model.layer1.bias

Parameter containing:
tensor([-0.6358,  0.7535], requires_grad=True)

In [35]:
weight_ = my_model.layer1.weight

bias_ = my_model.layer1.bias

In [46]:
w = torch.transpose(weight_, 0,1)

In [47]:
w

tensor([[-0.4598,  0.0734],
        [ 0.5194, -0.3372]], grad_fn=<TransposeBackward0>)

In [49]:
z = torch.matmul(test_my, w) + bias_

In [50]:
z

tensor([[-0.6163,  0.4707]], grad_fn=<AddBackward0>)

In [51]:
F.softmax(z)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.2522, 0.7478]], grad_fn=<SoftmaxBackward0>)